In [1]:
import numpy as np
from scipy.sparse import csr_matrix

def load_and_process_data_to_sparse_matrix(filename):
    # Initialize lists for class labels and feature entries
    labels = []
    row_indices = []
    col_indices = []
    data_values = []
    
    # Open and read the file line by line
    with open(filename, 'r') as file:
        row_index = 0
        for line in file:
            parts = line.split()
            labels.append(int(parts[0]))  # The first part is the class label
            
            for feature in parts[1:]:
                index, value = feature.split(':')
                col_indices.append(int(index))
                data_values.append(float(value))
                row_indices.append(row_index)
            
            row_index += 1
    
    # Determine the shape of the matrix
    num_rows = row_index
    num_cols = max(col_indices) + 1  # Adding 1 because indices start at 0
    
    # Create the CSR matrix from the row, column, and data value lists
    X_sparse = csr_matrix((data_values, (row_indices, col_indices)), shape=(num_rows, num_cols))
    
    # Convert labels list to a numpy array
    Y = np.array(labels)
    
    return Y, X_sparse

# Load and process the data from the 'ist' file into sparse format
Y, X_sparse = load_and_process_data_to_sparse_matrix('ledgar_lexglue_tfidft_trainx')
print("Class Labels (Y):", Y)
print("Features Matrix (X):", X_sparse.shape)
print(np.unique(Y))

Class Labels (Y): [97 39 26 45 11 65 73 50 98 47  6  7 43 58 19 47 93 38 65 79 96 85 66 26
 13 92 89 58 93 20  7 38 38 26 58  2 97 47 79 95 71 46 76 99 85 79 62  7
 84 47 47  7 26 75 87 54 39 26 28 55 47 17 41  2 26 44 79 90 47 74 39 84
 65 81  2 73 46 51 41  6 31 26 23 35 33 79 96 65 92 45  7 56 20 85 47 19
 42  4 50 68 26 23 15 67 22 28 83 53 16 88 38 25 32 35 65 66 59 26 38 92
 38 26 40  7 38 32  2 85 51 38 96 31 95 91 66 99 20 41 66 47 65 32 53 49
 26 19 59 18 65 45 16 97 31 93 41 12 79 38 15 38 24 88 79 91 51 88 85 42
  6 88 27 69 38 22 65 97 32 74 38 68 79  9 38 88 59 24 41 49 68 20 22 89
 83 73 85 75 10 17 57 47 66 38  1 28 26 41 10  5 19 88 77 42 26 22 17 40
 51 41 65 29 33 65 99 97 93  6 89 77 11 41 29 37 65 92 49 84 92 76 48 50
 61 19 87 58 81  7 18 58 10  6 38 31 83 79 55 82 53 26 29 46 65  7 26 20
 38 89 58 81 54 56 89 45 20 26 26 38 16 80 65 35 94 13 47 84 25 10 33 38
 97 54  2 13 55 65 65 65 74 26 43 10 31 12 42 47 58 10 95 61 18 24 19  7
 19 85  2 20 65 32 80 18 26 31  2

In [5]:
def select_5_percent_samples(X, y, seed):
    
    np.random.seed(seed)  ## Seed has been fixed to 42 in the beginning of the code.

    # Determine the number of samples to select from each class (5%)
    num_samples_per_class = {label: int(np.ceil(0.05 * np.sum(y == label))) for label in np.unique(y)}

    ## For every unique class, I choose the (ceiling_%5) amount of instances. 
    ## I tried using floor function but the instances per class came out to be too low to get meaningful results.

    # Initialize arrays to store the indices of labeled and unlabeled data
    labeled_indices = np.array([], dtype=int)
    unlabeled_indices = np.array([], dtype=int)

    # Select 5% from each class as labeled data
    for label in np.unique(y):
        indices = np.where(y == label)[0] 
        
## This will give me the indices of the instances which belong to the class 'label', which is a number from 0 to 2.
        np.random.shuffle(indices)
        label_indices = indices[:num_samples_per_class[label]]
        unlabeled_indices = np.concatenate((unlabeled_indices, indices[num_samples_per_class[label]:]))
        labeled_indices = np.concatenate((labeled_indices, label_indices))

    # Split the data into labeled and unlabeled sets
    X_labeled = X[labeled_indices]
    y_labeled = y[labeled_indices]
    
    X_unlabeled = X[unlabeled_indices]
    y_unlabeled = y[unlabeled_indices]
    
    return X_labeled, y_labeled, X_unlabeled, y_unlabeled

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [6]:
alpha = 0.5  ## Damping factor
teta = 0.01  ## Treshold value to determine the strong affinities.
## These two paramerer values are also mentioned in the paper, p.7 
from sklearn.neighbors import NearestNeighbors
k = 6 ## As sugested in the paper, page 7

def construct_neighborhood_indicator_matrix(X):

    # Initialize the nearest neighbors model and fit it to the labeled data
    nn = NearestNeighbors(algorithm='auto', metric="euclidean").fit(X)

    distances, indices = nn.kneighbors(X) ## For this matrix, we are not interested in distances, only indices.

    # Initialize the neighborhood indicator matrix P with zeros
    P = np.zeros((len(X), len(X)))

    # Fill in the neighborhood indicator matrix P for the labeled samples
    for i in range(len(X)):
        P[i, indices[i]] = 1 / k
    
    return P


def affinity_propagation(P, y_labeled, alpha, teta):
    # Initialize the affinity matrix W_0
    n_samples = P.shape[0]
    W_0 = np.zeros((n_samples, n_samples)) ## np.eye(n_samples) ## da olabilir. Tekrar iyi bak.
    
    ## Initially creates a matix full of zeros with the respective dimensions
    
    labeled_indices = np.where(y_labeled != -1)[0]

    # Set affinities for similar and dissimilar pairs
    for i, idx_i in enumerate(labeled_indices):
        for j, idx_j in enumerate(labeled_indices):
            if y_labeled[i] == y_labeled[j]: ## Initially we mark similar points as if in 
                W_0[idx_i, idx_j] = 1  ## Similar pairs, as written in the paper
            else:
                W_0[idx_i, idx_j] = -1  ## Dissimilar pairs
    
    ## W0 has been created using %5 labeled indices. 
    ## Now we need to propagate the affinities through the neighborhood structure.

    # Propagate affinities through the neighborhood structure
    W = np.zeros_like(W_0) ##Creates an empty matrix with the same dimensions as W_0.

    for _ in range(n_samples):
        W = (1 - alpha) * W_0 + alpha * np.dot(P, W)
        
        # Apply the threshold to determine strong affinities
        W[np.abs(W) < teta] = 0
    
    return W

def step_2(W,X):

    W1 = np.sum(W, axis=1)  # This computes the sum of each row (axis=1 for rows)
    D = np.diag(W1)  # This creates a diagonal matrix D from the vector W1

    # Compute the graph Laplacian L
    L = D - W

    # Compute the matrix T
    T = X.T @ L @ X  ##  @ stands for matrix multiplication in numpy

    return T

In [7]:
# Use the function to perform affinity propagation
alpha = 0.5  # Example value for the damping factor
threshold = 0.01  # Example threshold value for strong affinities
X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X, Y, seed_value)

# Construct neighborhood indicator matrix P for the labeled data
P = construct_neighborhood_indicator_matrix(X)

print("P: ", P)  # Show the constructed matrix P for verification
row_sums = np.sum(P, axis=1)

W = affinity_propagation(P, y_labeled, alpha, teta)
T = step_2(W,X) ## It is basically an (n * n) square matrix where n = #features of X

## Verify the affinity matrix

print("W: ",W)  
print("T: ", T)

print("P shape: ", P.shape)
print("W shape: ", W.shape)
print("T shape: ", T.shape)

P:  [[0.16666667 0.         0.         ... 0.         0.         0.        ]
 [0.         0.16666667 0.         ... 0.         0.         0.        ]
 [0.         0.         0.16666667 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.16666667 0.         0.        ]
 [0.         0.         0.         ... 0.         0.16666667 0.        ]
 [0.         0.         0.         ... 0.         0.         0.16666667]]
W:  [[ 0.54325285 -0.54765624 -0.54765624 ...  0.          0.
   0.        ]
 [-0.70968956  0.41861602 -0.70968956 ...  0.          0.
   0.        ]
 [-0.67470073 -0.67470073  0.43902451 ...  0.          0.
   0.        ]
 ...
 [-0.06816256 -0.06816256 -0.06816256 ...  0.          0.
   0.        ]
 [-0.18513574 -0.18513574 -0.18513574 ...  0.          0.
   0.        ]
 [-0.06193811 -0.06086964 -0.06193811 ...  0.          0.
   0.        ]]
T:  [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.         -1.12303225  

In [8]:
def h_sigma(M, rho, sigma):
    # Apply Nesterov's smoothing technique
    ## to smooth the l1 term 
    U_star = np.minimum(rho, np.maximum(M / sigma, -rho))
    
    return np.trace(U_star.T @ M) - (sigma / 2) * np.linalg.norm(U_star, 'fro')**2

def grad_h_sigma(M, rho, sigma):
    return np.minimum(rho, np.maximum(M / sigma, -rho))

def f(M, Sigma):
    fM = -np.log(np.linalg.det(M)) + np.trace(Sigma @ M)
    return fM

def grad_f(M,Σ):
    grad_fM = -np.linalg.inv(M) + Σ
    return grad_fM

def Σ(X,y_labeled, alpha, beta):
    M0 = np.eye(X.shape[1])  # Initialize M0 as the identity matrix
    P = construct_neighborhood_indicator_matrix(X)
    W = affinity_propagation(P, y_labeled, alpha, teta)
    T = step_2(W)
    Σ = np.linalg.inv(M0) + beta * T  
    return Σ

In [9]:
def ALM(M_init, Y_init, mu, rho, sigma, Σ, max_iter=500):
    M = M_init
    Y = Y_init
    
    for _ in range(max_iter):

        # Step 1: Update M
        grad_h_Y = grad_h_sigma(Y, rho, sigma)
        M_next = M - mu * (grad_h_Y + grad_f(M,Σ))
        
        # Step 2: Update Y
        grad_f_M_next = grad_f(M_next,Σ)
        Y_next = Y - mu * (grad_f_M_next + grad_h_sigma(M_next, rho, sigma))
        
        if np.linalg.norm(M_next - M, 'fro') < 1e-6:
            break
         
        # Prepare for next iteration
        M = M_next
        Y = Y_next
    return M

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

rho = 10000 ## Tuning (smoothness parameter)
beta = 2   ## Tuning parameter
mu = 0.000001  # Update step size
sigma = 0.000001   # Smoothness parameter

def run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta):

    # Construct neighborhood indicator matrix P for the labeled data
    P = construct_neighborhood_indicator_matrix(X)
    # Apply affinity propagation to obtain the affinity matrix W
    W = affinity_propagation(P, y_labeled, alpha, teta)
    # Compute matrix T
    T = step_2(W,X)

    # Set Σ = np.linalg.inv(M0) + beta * T  
    Σ = np.linalg.inv(M0) + beta * T

    # Run ALM to obtain the sparse metric matrix M
    M = ALM( M0, Y0, mu, rho, sigma, Σ)
    
    knn = KNeighborsClassifier(n_neighbors=1, metric="mahalanobis" , metric_params={'VI': np.linalg.inv(M)})
    knn.fit(X_labeled, y_labeled) ## Model training phase with the help of labeled data points.

    # Predict labels for unlabeled data
    y_pred = knn.predict(X_unlabeled)
    # Calculate accuracy score and error rate
    accuracy = accuracy_score(y_unlabeled, y_pred)
    error_rate = 1 - accuracy
    print("S3ML Error rate: ", error_rate)
    return error_rate

In [ ]:
def experiment_S3ML(experiment_number,  seed_value,mu, rho, sigma, beta, avg_S3ML_error_rate):
    for _ in range(experiment_number):
        X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X, Y, seed_value)
        M0 = np.identity(X.shape[1])
        Y0 = np.zeros_like(M0)

        avg_S3ML_error_rate += run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
        seed_value += 1
        
    avg_S3ML_error_rate = avg_S3ML_error_rate/experiment_number

    print("Average error rate of S3ML after ", experiment_number ,"tests: ", avg_S3ML_error_rate)

experiment_S3ML(1,  seed_value, mu, rho, sigma, beta,0)

: 

In [ ]:
###     Grid search (extra):
def search_optimal_parameters(X, y, seed_value, mu, sigma, start_value=1, max_value=1000, experiment_number=1):
    best_rho = start_value
    best_beta = start_value
    best_error_rate = float('inf')

    rho = start_value
    beta = start_value

    while rho <= max_value or beta <= max_value:
        avg_error_rate = 0
        for _ in range(experiment_number):
            X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X, y, seed_value)
            M0 = np.identity(X.shape[1])
            Y0 = np.zeros_like(M0)

            error_rate = run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
            seed_value += 1
            avg_error_rate += error_rate

        avg_error_rate /= experiment_number

        print(f"rho: {rho}, beta: {beta}, avg accuracy rate: {1- avg_error_rate}")

        if avg_error_rate < best_error_rate:
            best_error_rate = avg_error_rate
            best_rho = rho
            best_beta = beta

        if rho <= max_value:
            rho *= 2

        elif beta <= max_value:
            beta *= 2
            rho = 1  # Reset rho to 1 when beta increases

        if rho > max_value and beta > max_value:
            break

    return best_rho, best_beta, best_error_rate

# Initial parameters
mu = 1e-6
sigma = 1e-6
seed_value = 42

best_rho, best_beta, best_error_rate = search_optimal_parameters(X, Y, seed_value, mu, sigma)
print(f"Best rho: {best_rho}, Best beta: {best_beta}, Best accuracy rate: {1- best_error_rate}")

In [ ]:
## Random search (extra):
from sklearn.model_selection import KFold

def random_search_s3ml(X, y, folds, num_samples, rho_range, beta_range, mu, sigma, seed_value):
    np.random.seed(seed_value)
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed_value)
    best_rho = None
    best_beta = None
    best_error_rate = float('inf')

    for _ in range(num_samples):
        rho = np.random.uniform(*rho_range)
        beta = np.random.uniform(*beta_range)
        error_rates = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X_train, y_train, seed_value)
            M0 = np.identity(X.shape[1])
            Y0 = np.zeros_like(M0)

            error_rate = run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
            error_rates.append(error_rate)
            seed_value += 1

        avg_error_rate = np.mean(error_rates)
        if avg_error_rate < best_error_rate:
            best_error_rate = avg_error_rate
            best_rho = rho
            best_beta = beta

        print(f"Sampling: rho={rho:.2f}, beta={beta:.2f}, Avg Error Rate={avg_error_rate:.4f}")

    return best_rho, best_beta, best_error_rate

rho_range = (1e-2, 1e4)  
beta_range = (1e-2, 1e2)  

mu = 1e-6
sigma = 1e-6
seed_value = 42
folds = 10
num_samples = 100  # Number of random samples to test

best_rho, best_beta, best_error_rate = random_search_s3ml(X, y, folds, num_samples, rho_range, beta_range, mu, sigma, seed_value)
print(f"Best rho: {best_rho}, Best beta: {best_beta}, Best error rate: {best_error_rate:.4f}")

In [ ]:
## Grid search based KFold cross-validation (extra):
from sklearn.model_selection import KFold

def KFoldC_s3ml(X, y, folds, rho_values, beta_values, mu, sigma, seed_value):
    kf = KFold(n_splits= folds, shuffle= True, random_state= seed_value)
    best_rho ,best_beta ,best_error_rate = None, None, float('inf')

    for rho in rho_values:
        for beta in beta_values:
            error_rates = []
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                X_labeled, y_labeled, X_unlabeled, y_unlabeled = select_5_percent_samples(X_train, y_train, seed_value)
                M0 = np.identity(X.shape[1])
                Y0 = np.zeros_like(M0)

                error_rate = run_s3ml(X_labeled, y_labeled, X_unlabeled, y_unlabeled, M0, Y0, mu, rho, sigma, beta)
                error_rates.append(error_rate)
                seed_value += 1

            avg_error_rate = np.mean(error_rates)
            if avg_error_rate < best_error_rate:
                best_error_rate = avg_error_rate
                best_rho = rho
                best_beta = beta

            print(f"Grid Search: rho={rho:.2f}, beta={beta:.2f}, Avg Error Rate={avg_error_rate:.4f}")

    return best_rho, best_beta, best_error_rate

rho_values = np.logspace(-2, 4, num=10)  # Logarithmically spaced values from 0.01 to 10000
beta_values = np.logspace(-2, 2, num=10)  # Logarithmically spaced values from 0.01 to 100

mu = 1e-6
sigma = 1e-6
seed_value = 42
folds = 10

best_rho, best_beta, best_error_rate = KFoldC_s3ml(X, y, folds, rho_values, beta_values, mu, sigma, seed_value)
print(f"Best rho: {best_rho}, Best beta: {best_beta}, Best error rate: {best_error_rate:.4f}")
